In [1]:
import pandas as pd

In [5]:
import pandas as pd
import numpy as np

index = "Ind"
title = "title"
article = "text"
ground_truth = "summary"

files = [
    "mbart_predictions.txt",
    "mt5_predictions.txt",
    "summarunner_predictions.txt",
    "llama_7b_predictions.csv",
    "starling_predictions.csv",
    "yagpt_predictions.csv",
    "yagpt3_predictions.csv"
]
summaries_fields = []
human_metrics = [
    "Актуальность",
    "Последовательность",
    "Беглость",
    "Согласованность",
    "Комментарий"
]

for file in files:
    model_name = file.split(".")[0]
    summaries_fields.append(model_name)
    for metric in human_metrics:
        summaries_fields.append(f"{model_name}_{metric}")

summaries_fields_types = {field : ('Float64' if field.split("_")[-1] != human_metrics[-1] and field + ".txt" not in files and field + ".csv" not in files else str) for field in summaries_fields}
print(summaries_fields_types)
summaries_fields_types[index] = 'Int64'
summaries_fields_types[article] = summaries_fields_types[ground_truth] = str

metrics_data = pd.read_csv("metrics_data.csv", dtype=summaries_fields_types)
expert_data = pd.read_csv("compiled_expert_data.csv", dtype=summaries_fields_types)

{'mbart_predictions': <class 'str'>, 'mbart_predictions_Актуальность': 'Float64', 'mbart_predictions_Последовательность': 'Float64', 'mbart_predictions_Беглость': 'Float64', 'mbart_predictions_Согласованность': 'Float64', 'mbart_predictions_Комментарий': <class 'str'>, 'mt5_predictions': <class 'str'>, 'mt5_predictions_Актуальность': 'Float64', 'mt5_predictions_Последовательность': 'Float64', 'mt5_predictions_Беглость': 'Float64', 'mt5_predictions_Согласованность': 'Float64', 'mt5_predictions_Комментарий': <class 'str'>, 'summarunner_predictions': <class 'str'>, 'summarunner_predictions_Актуальность': 'Float64', 'summarunner_predictions_Последовательность': 'Float64', 'summarunner_predictions_Беглость': 'Float64', 'summarunner_predictions_Согласованность': 'Float64', 'summarunner_predictions_Комментарий': <class 'str'>, 'llama_7b_predictions': <class 'str'>, 'llama_7b_predictions_Актуальность': 'Float64', 'llama_7b_predictions_Последовательность': 'Float64', 'llama_7b_predictions_Бегло

In [20]:
import math


models = {
    "mbart_predictions",
    "mt5_predictions",
    "summarunner_predictions",
    "llama_7b_predictions",
    "starling_predictions",
    "yagpt_predictions",
    "yagpt3_predictions"
}
metrics = {
    "bleu",
    "rouge1",
    "meteor",
    "bertscore_f1"
}


new_data = {"text": [], "summary": [], "machine_summary": [], "Актуальность": [], "Последовательность": [], "Беглость": [], "Согласованность": [], "bertscore_f1": [], "bleu": [], "rouge1": [], "meteor": []}


for model in models:
    for (mteric_index, metric_row), (expert_index, expert_row)  in zip(metrics_data.iterrows(), expert_data.iterrows()):
        # if any([row[f"{model}_{metric}"] is None or math.isnan(row[f"{model}_{metric}"]) for metric in metrics]) or row[model] is None or type(row[model]) != str:
        #     continue
        new_data["text"].append(metric_row["text"])
        new_data["summary"].append(metric_row["summary"])
        new_data["machine_summary"].append(metric_row[model])

        new_data["bleu"].append(metric_row[f"{model}_bleu"])
        new_data["rouge1"].append(metric_row[f"{model}_rouge1"])
        new_data["meteor"].append(metric_row[f"{model}_meteor"])
        new_data["bertscore_f1"].append(metric_row[f"{model}_bertscore_f1"])

        new_data["Актуальность"].append(expert_row[f"{model}_Актуальность"])
        new_data["Последовательность"].append(expert_row[f"{model}_Последовательность"])
        new_data["Беглость"].append(expert_row[f"{model}_Беглость"])
        new_data["Согласованность"].append(expert_row[f"{model}_Согласованность"])
        

In [21]:
print(len(new_data["text"]))

700


In [22]:
new_data = pd.DataFrame(new_data)

In [23]:
new_data.head()

,text,summary,machine_summary,Актуальность,Последовательность,Беглость,Согласованность,bertscore_f1,bleu,rouge1,meteor
0,Постоянно включенный Bluetooth на смартфоне гр...,Активированный в смартфоне Bluetooth может пре...,Постоянно включенный Bluetooth на смартфоне гр...,4.0,4.0,5.0,5.0,0.648150,0.036231,0.228070,0.126459
1,Член президиума координационного совета оппози...,Члена президиума оппозиционного Координационно...,Член президиума координационного совета оппози...,4.0,4.0,5.0,5.0,0.748946,0.082445,0.368932,0.248829
2,Аналитики Deutsche Bank обнародовали исследова...,В истории человечества наступает эпоха беспоря...,В 2020 году в мире наступит новая эпоха беспор...,5.0,5.0,5.0,5.0,0.742476,0.133905,0.395604,0.309631
3,ГИБДД совместно с Минтрансом разработала масшт...,Масштабный проект изменений в ПДД подготовили ...,ГИБДД совместно с Минтрансом разработала масшт...,5.0,5.0,4.0,5.0,0.709359,0.059402,0.333333,0.306315
4,Вечером 22 сентября на конференции Battery Day...,Миллиардер Илон Маск пообещал представить бюдж...,На конференции Battery Day глава Tesla Илон Ма...,5.0,5.0,5.0,5.0,0.643236,0.031277,0.175824,0.098891


## Concatenate

In [24]:
translated_data = pd.read_csv("data_and_metrics.csv")

In [27]:
print(translated_data.columns)
print(new_data.columns)

Index(['text', 'summary', 'machine_summary', 'Актуальность',
       'Последовательность', 'Беглость', 'Согласованность', 'bertscore_f1',
       'bleu', 'rouge1', 'meteor'],
      dtype='object')
Index(['text', 'summary', 'machine_summary', 'Актуальность',
       'Последовательность', 'Беглость', 'Согласованность', 'bertscore_f1',
       'bleu', 'rouge1', 'meteor'],
      dtype='object')


In [25]:
data = pd.concat([translated_data, new_data])
data.head()

,text,summary,machine_summary,Актуальность,Последовательность,Беглость,Согласованность,bertscore_f1,bleu,rouge1,meteor
0,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...","Дональд Стерлинг, игрок прошлогодней команды Н...",1.666667,1.000000,1.000000,1.333333,0.688328,0.101056,0.300885,0.270348
1,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...","Дональд Стерлинг обвинил Стивиано в том, что о...",1.666667,2.333333,4.666667,3.000000,0.666818,0.055018,0.297872,0.352888
2,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...",судья Лос-Анджелеса обязал В. Стивиано выплати...,2.333333,4.666667,4.333333,1.000000,0.650881,0.080116,0.285714,0.214511
3,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...",Жена Дональда Стерлинга подала в суд на Стивиа...,3.333333,5.000000,4.333333,2.666667,0.684118,0.060453,0.329897,0.370017
4,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...",расистские высказывания Дональда Стерлинга в п...,3.000000,5.000000,5.000000,4.666667,0.676089,0.109052,0.297297,0.226064


In [26]:
data.tail()

,text,summary,machine_summary,Актуальность,Последовательность,Беглость,Согласованность,bertscore_f1,bleu,rouge1,meteor
695,"Пройдитесь щеткой или расческой по волосам, уб...",Расчешите волосы расческой или щеткой. Выделит...,"чтобы завитки держались дольше, опрыскайте их ...",3.0,3.0,5.0,5.0,0.635249,0.066698,0.275862,0.134228
696,"Существует множество видов аквариумных рыб, и ...",По возможности расспросите ветеринара о состоя...,"пропустите данный шаг, если вы не будете испол...",2.0,2.0,5.0,4.0,0.635633,0.044284,0.109589,0.063613
697,Роющие осы – это насекомые очень больших разме...,"Узнайте больше, о роющих осах. Определите приз...","они превращаются из маленьких ос, играющих дру...",4.0,5.0,5.0,4.0,0.600029,0.037330,0.212389,0.139480
698,Тренируйте мышцы и сердце как минимум по 30 ми...,Занимайтесь спортом. Гуляйте на природе. Спите...,внесите изменения в свой привычный график. что...,4.0,3.0,5.0,5.0,0.648210,0.035197,0.123077,0.082988
699,Таиландские лабео с меньшей вероятностью будут...,Выберите в качестве соседей достаточно агресси...,наведайтесь в зоомагазин и выберите подходящие...,2.0,2.0,5.0,4.0,0.607463,0.037405,0.171429,0.125899


In [18]:
print(len(data))

2300


In [ ]:
data.to_csv("final_combined_data.csv", index=False)

In [ ]:
mean_human = np.mean(human_scores)
mean_bleu = np.mean(auto_scores["bleu"])
mean_rouge = np.mean(auto_scores["rouge"])
mean_meteor = np.mean(auto_scores["meteor"])
mean_bertscore = np.mean(auto_scores["bertscore"])

orig_scores["bleu"] = auto_scores["bleu"].copy()
orig_scores["rouge"] = auto_scores["rouge"].copy()
orig_scores["meteor"] = auto_scores["meteor"].copy()
orig_scores["bertscore"] = auto_scores["bertscore"].copy()

for i in range(len(texts)):
    human_scores[i] -= mean_human
    auto_scores["bleu"][i] -= mean_bleu
    auto_scores["rouge"][i] -= mean_rouge
    auto_scores["meteor"][i] -= mean_meteor
    auto_scores["bertscore"][i] -= mean_bertscore

    deviations["bleu"].append(abs(auto_scores["bleu"][i] - human_scores[i]))
    deviations["rouge"].append(abs(auto_scores["rouge"][i] - human_scores[i]))
    deviations["meteor"].append(abs(auto_scores["meteor"][i] - human_scores[i]))
    deviations["bertscore"].append(abs(auto_scores["bertscore"][i] - human_scores[i]))

print("Median devs:")
print("\tBLEU: ", np.median(deviations["bleu"]), np.max(deviations["bleu"]))
print("\tROUGE: ", np.median(deviations["rouge"]), np.max(deviations["rouge"]))
print("\tMETEOR: ", np.median(deviations["meteor"]), np.max(deviations["meteor"]))
print("\tBERTSCORE: ", np.median(deviations["bertscore"]), np.max(deviations["bertscore"]))

labels["bleu"] = [1.0 if deviations["bleu"][i] < np.quantile(deviations["bleu"], 0.5) else 0.0 for i in range(len(texts))]
labels["rouge"] = [1.0 if deviations["rouge"][i] < np.quantile(deviations["rouge"], 0.5) else 0.0 for i in range(len(texts))]
labels["meteor"] = [1.0 if deviations["meteor"][i] < np.quantile(deviations["meteor"], 0.5) else 0.0 for i in range(len(texts))]
labels["bertscore"] = [1.0 if deviations["bertscore"][i] < np.quantile(deviations["bertscore"], 0.5) else 0.0  for i in range(len(texts))]